In [1]:
from pyspark.sql import SparkSession

### Creating APP

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Bangladesh India Cricket") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

##### Reading dataset

In [5]:
dataset = spark.read.csv('/home/hasan/DATA SET/datasets_33080_43333_car data.csv', header=True)


In [6]:
#head of the dataset
dataset.show()


+-------------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|     Car_Name|Year|Selling_Price|Present_Price|Kms_Driven|Fuel_Type|Seller_Type|Transmission|Owner|
+-------------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|         ritz|2014|         3.35|         5.59|     27000|   Petrol|     Dealer|      Manual|    0|
|          sx4|2013|         4.75|         9.54|     43000|   Diesel|     Dealer|      Manual|    0|
|         ciaz|2017|         7.25|         9.85|      6900|   Petrol|     Dealer|      Manual|    0|
|      wagon r|2011|         2.85|         4.15|      5200|   Petrol|     Dealer|      Manual|    0|
|        swift|2014|          4.6|         6.87|     42450|   Diesel|     Dealer|      Manual|    0|
|vitara brezza|2018|         9.25|         9.83|      2071|   Diesel|     Dealer|      Manual|    0|
|         ciaz|2015|         6.75|         8.12|     18796|   Petrol|     Dealer|      Manu

In [7]:
# checking data type and null value of the column
dataset.printSchema()


root
 |-- Car_Name: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Selling_Price: string (nullable = true)
 |-- Present_Price: string (nullable = true)
 |-- Kms_Driven: string (nullable = true)
 |-- Fuel_Type: string (nullable = true)
 |-- Seller_Type: string (nullable = true)
 |-- Transmission: string (nullable = true)
 |-- Owner: string (nullable = true)



#### Canging data type of the column

In [8]:
from pyspark.sql.types import DoubleType, IntegerType

dataset = dataset.withColumn("Selling_Price",dataset["Selling_Price"].cast(DoubleType()))
dataset = dataset.withColumn("Year",dataset["Year"].cast(IntegerType()))
dataset = dataset.withColumn("Present_Price",dataset["Present_Price"].cast(DoubleType()))
dataset = dataset.withColumn("Kms_Driven",dataset["Kms_Driven"].cast(IntegerType()))
dataset = dataset.withColumn("Owner",dataset["Owner"].cast(IntegerType()))


In [9]:
dataset.printSchema()


root
 |-- Car_Name: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Selling_Price: double (nullable = true)
 |-- Present_Price: double (nullable = true)
 |-- Kms_Driven: integer (nullable = true)
 |-- Fuel_Type: string (nullable = true)
 |-- Seller_Type: string (nullable = true)
 |-- Transmission: string (nullable = true)
 |-- Owner: integer (nullable = true)



##### Checking null value in every column

In [10]:
from pyspark.sql.functions import isnan, when, count, col

dataset.select([count(when(isnan(c), c)).alias(c) for c in dataset.columns]).show()


+--------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|Car_Name|Year|Selling_Price|Present_Price|Kms_Driven|Fuel_Type|Seller_Type|Transmission|Owner|
+--------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|       0|   0|            0|            0|         0|        0|          0|           0|    0|
+--------+----+-------------+-------------+----------+---------+-----------+------------+-----+



##### Selecting column

In [11]:
#selecting a single column
dataset.select('Fuel_Type').show()


+---------+
|Fuel_Type|
+---------+
|   Petrol|
|   Diesel|
|   Petrol|
|   Petrol|
|   Diesel|
|   Diesel|
|   Petrol|
|   Diesel|
|   Diesel|
|   Diesel|
|   Petrol|
|   Diesel|
|   Petrol|
|   Petrol|
|   Petrol|
|   Diesel|
|   Diesel|
|   Diesel|
|      CNG|
|   Petrol|
+---------+
only showing top 20 rows



In [12]:
# Selecting multiple column
dataset.select(['Car_Name', 'Fuel_Type']).show()


+-------------+---------+
|     Car_Name|Fuel_Type|
+-------------+---------+
|         ritz|   Petrol|
|          sx4|   Diesel|
|         ciaz|   Petrol|
|      wagon r|   Petrol|
|        swift|   Diesel|
|vitara brezza|   Diesel|
|         ciaz|   Petrol|
|      s cross|   Diesel|
|         ciaz|   Diesel|
|         ciaz|   Diesel|
|     alto 800|   Petrol|
|         ciaz|   Diesel|
|         ciaz|   Petrol|
|       ertiga|   Petrol|
|        dzire|   Petrol|
|       ertiga|   Diesel|
|       ertiga|   Diesel|
|       ertiga|   Diesel|
|      wagon r|      CNG|
|          sx4|   Petrol|
+-------------+---------+
only showing top 20 rows



##### Selecting column with value chagne

In [13]:
#original value
dataset.select(['Transmission', 'Owner']).show(5)


+------------+-----+
|Transmission|Owner|
+------------+-----+
|      Manual|    0|
|      Manual|    0|
|      Manual|    0|
|      Manual|    0|
|      Manual|    0|
+------------+-----+
only showing top 5 rows



In [14]:
#with changed value
dataset.select(dataset['Transmission'], dataset['Owner']+1).show(5)


+------------+-----------+
|Transmission|(Owner + 1)|
+------------+-----------+
|      Manual|          1|
|      Manual|          1|
|      Manual|          1|
|      Manual|          1|
|      Manual|          1|
+------------+-----------+
only showing top 5 rows



##### Filterring data

In [15]:
# single filtering
dataset.filter(dataset['Year']>2017).show()


+-------------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|     Car_Name|Year|Selling_Price|Present_Price|Kms_Driven|Fuel_Type|Seller_Type|Transmission|Owner|
+-------------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|vitara brezza|2018|         9.25|         9.83|      2071|   Diesel|     Dealer|      Manual|    0|
+-------------+----+-------------+-------------+----------+---------+-----------+------------+-----+



##### Filtering with &

In [16]:
#multiple filtering search
dataset.filter((dataset['Year']>2016) & (dataset['Seller_Type']=='Dealer')).show()


+-------------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|     Car_Name|Year|Selling_Price|Present_Price|Kms_Driven|Fuel_Type|Seller_Type|Transmission|Owner|
+-------------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|         ciaz|2017|         7.25|         9.85|      6900|   Petrol|     Dealer|      Manual|    0|
|vitara brezza|2018|         9.25|         9.83|      2071|   Diesel|     Dealer|      Manual|    0|
|     alto 800|2017|         2.85|          3.6|      2135|   Petrol|     Dealer|      Manual|    0|
|        ignis|2017|          4.9|         5.71|      2400|   Petrol|     Dealer|      Manual|    0|
|         ciaz|2017|         7.75|         9.29|     37000|   Petrol|     Dealer|   Automatic|    0|
|       innova|2017|         18.0|        19.77|     15000|   Diesel|     Dealer|   Automatic|    0|
|     fortuner|2017|         33.0|        36.23|      6000|   Diesel|     Dealer|   Automat

In [17]:
#multiple filtering search
dataset.filter((dataset['Year']>2016) & (dataset['Seller_Type']=='Dealer') & (dataset['Present_Price']>30)).show()


+--------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|Car_Name|Year|Selling_Price|Present_Price|Kms_Driven|Fuel_Type|Seller_Type|Transmission|Owner|
+--------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|fortuner|2017|         33.0|        36.23|      6000|   Diesel|     Dealer|   Automatic|    0|
+--------+----+-------------+-------------+----------+---------+-----------+------------+-----+



##### Filtering with or

In [18]:
# multiple filtering search
dataset.filter((dataset['Year']==2016) | (dataset['Selling_Price']==4)).show()


+--------------------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|            Car_Name|Year|Selling_Price|Present_Price|Kms_Driven|Fuel_Type|Seller_Type|Transmission|Owner|
+--------------------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|                ciaz|2016|         8.75|         8.89|     20273|   Diesel|     Dealer|      Manual|    0|
|              ertiga|2016|         7.75|        10.79|     43000|   Diesel|     Dealer|      Manual|    0|
|              ertiga|2016|         7.75|        10.79|     43000|   Diesel|     Dealer|      Manual|    0|
|            alto k10|2016|         2.85|         3.95|     25000|   Petrol|     Dealer|      Manual|    0|
|            alto k10|2016|          3.0|         3.76|     10079|   Petrol|     Dealer|      Manual|    0|
|              baleno|2016|         5.85|         7.87|     24524|   Petrol|     Dealer|   Automatic|    0|
|       corolla altis|2016| 

In [19]:
#multiple filtering search
dataset.filter((dataset['Year']>2020) | (dataset['Seller_Type']=='Dealer') | (dataset['Present_Price']==30)).show()


+-------------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|     Car_Name|Year|Selling_Price|Present_Price|Kms_Driven|Fuel_Type|Seller_Type|Transmission|Owner|
+-------------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|         ritz|2014|         3.35|         5.59|     27000|   Petrol|     Dealer|      Manual|    0|
|          sx4|2013|         4.75|         9.54|     43000|   Diesel|     Dealer|      Manual|    0|
|         ciaz|2017|         7.25|         9.85|      6900|   Petrol|     Dealer|      Manual|    0|
|      wagon r|2011|         2.85|         4.15|      5200|   Petrol|     Dealer|      Manual|    0|
|        swift|2014|          4.6|         6.87|     42450|   Diesel|     Dealer|      Manual|    0|
|vitara brezza|2018|         9.25|         9.83|      2071|   Diesel|     Dealer|      Manual|    0|
|         ciaz|2015|         6.75|         8.12|     18796|   Petrol|     Dealer|      Manu

##### Grouping

In [20]:
#single column grouping
dataset.groupBy('Fuel_Type').count().show()


+---------+-----+
|Fuel_Type|count|
+---------+-----+
|   Diesel|   60|
|      CNG|    2|
|   Petrol|  239|
+---------+-----+



In [21]:
# multiple column grouping
dataset.groupBy(['Fuel_Type', 'Transmission']).count().show()


+---------+------------+-----+
|Fuel_Type|Transmission|count|
+---------+------------+-----+
|   Petrol|   Automatic|   28|
|   Diesel|      Manual|   48|
|      CNG|      Manual|    2|
|   Petrol|      Manual|  211|
|   Diesel|   Automatic|   12|
+---------+------------+-----+



In [22]:
# multiple column grouping
dataset.groupBy(['Fuel_Type', 'Transmission']).sum().show()


+---------+------------+---------+------------------+------------------+---------------+----------+
|Fuel_Type|Transmission|sum(Year)|sum(Selling_Price)|sum(Present_Price)|sum(Kms_Driven)|sum(Owner)|
+---------+------------+---------+------------------+------------------+---------------+----------+
|   Petrol|   Automatic|    56368|128.41000000000003|244.90999999999997|        1482812|         3|
|   Diesel|      Manual|    96663|            368.32|            581.16|        2382488|         1|
|      CNG|      Manual|     4026|               6.2|             12.83|          85498|         0|
|   Petrol|      Manual|   424868| 651.7299999999997|           1089.56|        6530604|         9|
|   Diesel|   Automatic|    24177|            248.39|            367.71|         639707|         0|
+---------+------------+---------+------------------+------------------+---------------+----------+



### Using SQL Query

In [23]:
dataset = spark.read.csv('/home/hasan/DATA SET/datasets_33080_43333_car data.csv', header=True)


In [24]:
dataset.show(5)


+--------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|Car_Name|Year|Selling_Price|Present_Price|Kms_Driven|Fuel_Type|Seller_Type|Transmission|Owner|
+--------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|    ritz|2014|         3.35|         5.59|     27000|   Petrol|     Dealer|      Manual|    0|
|     sx4|2013|         4.75|         9.54|     43000|   Diesel|     Dealer|      Manual|    0|
|    ciaz|2017|         7.25|         9.85|      6900|   Petrol|     Dealer|      Manual|    0|
| wagon r|2011|         2.85|         4.15|      5200|   Petrol|     Dealer|      Manual|    0|
|   swift|2014|          4.6|         6.87|     42450|   Diesel|     Dealer|      Manual|    0|
+--------+----+-------------+-------------+----------+---------+-----------+------------+-----+
only showing top 5 rows



In [25]:
dataset.printSchema()


root
 |-- Car_Name: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Selling_Price: string (nullable = true)
 |-- Present_Price: string (nullable = true)
 |-- Kms_Driven: string (nullable = true)
 |-- Fuel_Type: string (nullable = true)
 |-- Seller_Type: string (nullable = true)
 |-- Transmission: string (nullable = true)
 |-- Owner: string (nullable = true)



In [26]:
# Register the DataFrame as a SQL temporary view
dataset.createOrReplaceTempView("car_dataset")


### How to select Column in SQL

In [27]:
#selecting all column
spark.sql("select * from car_dataset").show()


+-------------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|     Car_Name|Year|Selling_Price|Present_Price|Kms_Driven|Fuel_Type|Seller_Type|Transmission|Owner|
+-------------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|         ritz|2014|         3.35|         5.59|     27000|   Petrol|     Dealer|      Manual|    0|
|          sx4|2013|         4.75|         9.54|     43000|   Diesel|     Dealer|      Manual|    0|
|         ciaz|2017|         7.25|         9.85|      6900|   Petrol|     Dealer|      Manual|    0|
|      wagon r|2011|         2.85|         4.15|      5200|   Petrol|     Dealer|      Manual|    0|
|        swift|2014|          4.6|         6.87|     42450|   Diesel|     Dealer|      Manual|    0|
|vitara brezza|2018|         9.25|         9.83|      2071|   Diesel|     Dealer|      Manual|    0|
|         ciaz|2015|         6.75|         8.12|     18796|   Petrol|     Dealer|      Manu

In [28]:
#selecting a column
spark.sql("select Car_Name from car_dataset").show()


+-------------+
|     Car_Name|
+-------------+
|         ritz|
|          sx4|
|         ciaz|
|      wagon r|
|        swift|
|vitara brezza|
|         ciaz|
|      s cross|
|         ciaz|
|         ciaz|
|     alto 800|
|         ciaz|
|         ciaz|
|       ertiga|
|        dzire|
|       ertiga|
|       ertiga|
|       ertiga|
|      wagon r|
|          sx4|
+-------------+
only showing top 20 rows



In [29]:
#selecting two column
spark.sql("select Selling_Price,Present_Price from car_dataset").show()


+-------------+-------------+
|Selling_Price|Present_Price|
+-------------+-------------+
|         3.35|         5.59|
|         4.75|         9.54|
|         7.25|         9.85|
|         2.85|         4.15|
|          4.6|         6.87|
|         9.25|         9.83|
|         6.75|         8.12|
|          6.5|         8.61|
|         8.75|         8.89|
|         7.45|         8.92|
|         2.85|          3.6|
|         6.85|        10.38|
|          7.5|         9.94|
|          6.1|         7.71|
|         2.25|         7.21|
|         7.75|        10.79|
|         7.25|        10.79|
|         7.75|        10.79|
|         3.25|         5.09|
|         2.65|         7.98|
+-------------+-------------+
only showing top 20 rows



### Use of where in SQL

In [30]:
#selecting a specific value
spark.sql("select Year from car_dataset where Year==2014").show()


+----+
|Year|
+----+
|2014|
|2014|
|2014|
|2014|
|2014|
|2014|
|2014|
|2014|
|2014|
|2014|
|2014|
|2014|
|2014|
|2014|
|2014|
|2014|
|2014|
|2014|
|2014|
|2014|
+----+
only showing top 20 rows



In [31]:
#price in specific year
spark.sql("select Year,Present_Price from car_dataset where Year==2014").show()


+----+-------------+
|Year|Present_Price|
+----+-------------+
|2014|         5.59|
|2014|         6.87|
|2014|         3.46|
|2014|         7.49|
|2014|         9.95|
|2014|         8.06|
|2014|         3.98|
|2014|         8.06|
|2014|        12.04|
|2014|          6.8|
|2014|        35.96|
|2014|        35.96|
|2014|         6.95|
|2014|         6.76|
|2014|         8.93|
|2014|         6.76|
|2014|        16.09|
|2014|         3.45|
|2014|          2.4|
|2014|          1.2|
+----+-------------+
only showing top 20 rows



In [32]:
# advance filtering
spark.sql("select Year,Seller_Type from car_dataset where (Year==2014 AND Seller_Type=='Dealer')").show()


+----+-----------+
|Year|Seller_Type|
+----+-----------+
|2014|     Dealer|
|2014|     Dealer|
|2014|     Dealer|
|2014|     Dealer|
|2014|     Dealer|
|2014|     Dealer|
|2014|     Dealer|
|2014|     Dealer|
|2014|     Dealer|
|2014|     Dealer|
|2014|     Dealer|
|2014|     Dealer|
|2014|     Dealer|
|2014|     Dealer|
|2014|     Dealer|
|2014|     Dealer|
|2014|     Dealer|
|2014|     Dealer|
|2014|     Dealer|
|2014|     Dealer|
+----+-----------+
only showing top 20 rows



In [33]:
# advance filtering
spark.sql("select Year,Seller_Type,Kms_Driven from car_dataset where (Year==2015 AND Seller_Type=='Dealer' AND Kms_Driven>50000)").show()


+----+-----------+----------+
|Year|Seller_Type|Kms_Driven|
+----+-----------+----------+
|2015|     Dealer|     51000|
|2015|     Dealer|     61381|
|2015|     Dealer|     68000|
|2015|     Dealer|     60076|
|2015|     Dealer|     60000|
+----+-----------+----------+



### Statistical SQL

In [34]:
# summation of Present_Price
spark.sql("select SUM(Present_Price) from car_dataset").show()


+----------------------------------+
|sum(CAST(Present_Price AS DOUBLE))|
+----------------------------------+
|                2296.1700000000005|
+----------------------------------+



In [35]:
# Maximum of Present_Price
spark.sql("select MAX(Present_Price) from car_dataset").show()


+------------------+
|max(Present_Price)|
+------------------+
|              92.6|
+------------------+



In [36]:
# average of Present_Price
spark.sql("select AVG(Present_Price) from car_dataset").show()


+----------------------------------+
|avg(CAST(Present_Price AS DOUBLE))|
+----------------------------------+
|                 7.628471760797344|
+----------------------------------+



In [37]:
#printing distinct element of a column
spark.sql("select DISTINCT Fuel_Type from car_dataset ").show()


+---------+
|Fuel_Type|
+---------+
|   Diesel|
|      CNG|
|   Petrol|
+---------+



In [38]:
#printing distinct element of a column
spark.sql(" select SUM(Present_Price) from car_dataset where Year=2015 ").show()


+----------------------------------+
|sum(CAST(Present_Price AS DOUBLE))|
+----------------------------------+
|                 499.1000000000001|
+----------------------------------+



### Some Complex query

In [39]:
#printing price of some specific car
spark.sql(" select SUM(Present_Price) from car_dataset where Car_Name IN ('ritz','sx4','ciaz','wagon r','swift','vitara brezza') ").show()


+----------------------------------+
|sum(CAST(Present_Price AS DOUBLE))|
+----------------------------------+
|                216.89999999999995|
+----------------------------------+



In [40]:
#printing price of some specific car
spark.sql(" select Car_Name, SUM(Present_Price) from car_dataset where Present_Price>3 Group By Car_Name ").show()


+-------------+----------------------------------+
|     Car_Name|sum(CAST(Present_Price AS DOUBLE))|
+-------------+----------------------------------+
|corolla altis|                            281.32|
|      wagon r|                              17.8|
|  etios cross|                              23.9|
|        creta|                              40.8|
|          i10|                             22.83|
| land cruiser|                              92.6|
|       ertiga|                             57.74|
|         brio|                             60.44|
|        amaze|                 47.49999999999999|
|       baleno|                              7.87|
|         ciaz|                             87.81|
|        dzire|                             30.53|
|      s cross|                              8.61|
|      corolla|                             12.35|
|         city|                            306.59|
|          sx4|                              48.4|
|      etios g|                

In [41]:
# multiple column with a condition
spark.sql(" select Car_Name from car_dataset where Year=2014").show()


+--------------+
|      Car_Name|
+--------------+
|          ritz|
|         swift|
|      alto k10|
|         swift|
|        ertiga|
|         dzire|
|      alto k10|
|         dzire|
|          ciaz|
|       etios g|
|      fortuner|
|      fortuner|
|    etios liva|
|    etios liva|
|   etios cross|
|       etios g|
|        innova|
|Hyosung GT250R|
| KTM 390 Duke |
| Honda CBR 150|
+--------------+
only showing top 20 rows



In [42]:
# multiple column with a condition
spark.sql(" select (Present_Price - Selling_Price) as price_difference from car_dataset ").show(5)


+------------------+
|  price_difference|
+------------------+
|2.2399999999999998|
| 4.789999999999999|
|2.5999999999999996|
|1.3000000000000003|
|2.2700000000000005|
+------------------+
only showing top 5 rows



In [43]:
# filterring with column name
spark.sql("select max(Year) as maximum_year, count(Seller_Type) as seller_type_number from car_dataset").show()


+------------+------------------+
|maximum_year|seller_type_number|
+------------+------------------+
|        2018|               301|
+------------+------------------+



In [44]:
# filterring with column name
spark.sql("select max(Year) as maximum_year, count(Seller_Type) as seller_type_number from car_dataset where Fuel_Type=='Diesel' ").show()


+------------+------------------+
|maximum_year|seller_type_number|
+------------+------------------+
|        2018|                60|
+------------+------------------+



### use of concat

In [45]:
# adding two column
spark.sql("select concat(Car_Name,', ',Fuel_Type) as name_and_fuel_type from car_dataset").show()


+--------------------+
|  name_and_fuel_type|
+--------------------+
|        ritz, Petrol|
|         sx4, Diesel|
|        ciaz, Petrol|
|     wagon r, Petrol|
|       swift, Diesel|
|vitara brezza, Di...|
|        ciaz, Petrol|
|     s cross, Diesel|
|        ciaz, Diesel|
|        ciaz, Diesel|
|    alto 800, Petrol|
|        ciaz, Diesel|
|        ciaz, Petrol|
|      ertiga, Petrol|
|       dzire, Petrol|
|      ertiga, Diesel|
|      ertiga, Diesel|
|      ertiga, Diesel|
|        wagon r, CNG|
|         sx4, Petrol|
+--------------------+
only showing top 20 rows



### Use of In and Not-in

In [48]:
# filtering with name using in
spark.sql("select Car_Name,Fuel_Type from car_dataset where Car_Name in ('ciaz','ciaz','ertiga','dzire','ertiga','ertiga','ertiga','wagon r','sx4','alto k10','ignis','sx4','alto k10')").show(10)


+--------+---------+
|Car_Name|Fuel_Type|
+--------+---------+
|     sx4|   Diesel|
|    ciaz|   Petrol|
| wagon r|   Petrol|
|    ciaz|   Petrol|
|    ciaz|   Diesel|
|    ciaz|   Diesel|
|    ciaz|   Diesel|
|    ciaz|   Petrol|
|  ertiga|   Petrol|
|   dzire|   Petrol|
+--------+---------+
only showing top 10 rows



In [49]:
# filtering with name using not-in
spark.sql("select Car_Name,Fuel_Type from car_dataset where Car_Name not in ('ciaz','ciaz','ertiga','dzire','ertiga','ertiga','ertiga','wagon r','sx4','alto k10','ignis','sx4','alto k10')").show(10)


+-------------+---------+
|     Car_Name|Fuel_Type|
+-------------+---------+
|         ritz|   Petrol|
|        swift|   Diesel|
|vitara brezza|   Diesel|
|      s cross|   Diesel|
|     alto 800|   Petrol|
|        swift|   Petrol|
|        swift|   Petrol|
|        swift|   Petrol|
|         ritz|   Diesel|
|         ritz|   Petrol|
+-------------+---------+
only showing top 10 rows



### Use of Like

In [54]:
# filtering with name using not-in
spark.sql("select Car_Name,Fuel_Type from car_dataset where Car_Name like 'ci%' ").show(10)


+--------+---------+
|Car_Name|Fuel_Type|
+--------+---------+
|    ciaz|   Petrol|
|    ciaz|   Petrol|
|    ciaz|   Diesel|
|    ciaz|   Diesel|
|    ciaz|   Diesel|
|    ciaz|   Petrol|
|    ciaz|   Diesel|
|    ciaz|   Petrol|
|    ciaz|   Petrol|
|    city|   Petrol|
+--------+---------+
only showing top 10 rows



In [66]:
# filtering with name using not-in
spark.sql("select Car_Name,Fuel_Type from car_dataset where Car_Name like ('%z%' '%a%') ").show(10)


+-------------+---------+
|     Car_Name|Fuel_Type|
+-------------+---------+
|vitara brezza|   Diesel|
|Honda Karizma|   Petrol|
|Honda Karizma|   Petrol|
+-------------+---------+



In [62]:
# filtering with name using not-in
spark.sql("select Car_Name,Fuel_Type from car_dataset where Car_Name like ('%z' '%a') ").show(10)


+-------------+---------+
|     Car_Name|Fuel_Type|
+-------------+---------+
|vitara brezza|   Diesel|
|Honda Karizma|   Petrol|
|Honda Karizma|   Petrol|
+-------------+---------+



### Use of Between

In [68]:
# filtering with name using not-in
spark.sql("select Car_Name,Fuel_Type,Year from car_dataset where Year between 2014 and 2016 ").show(10)


+--------+---------+----+
|Car_Name|Fuel_Type|Year|
+--------+---------+----+
|    ritz|   Petrol|2014|
|   swift|   Diesel|2014|
|    ciaz|   Petrol|2015|
| s cross|   Diesel|2015|
|    ciaz|   Diesel|2016|
|    ciaz|   Diesel|2015|
|    ciaz|   Diesel|2015|
|    ciaz|   Petrol|2015|
|  ertiga|   Petrol|2015|
|  ertiga|   Diesel|2016|
+--------+---------+----+
only showing top 10 rows



### sorting 

In [69]:
# filtering with name using not-in
spark.sql("select Car_Name,Fuel_Type,Year from car_dataset order by Year ").show(10)


+-------------------+---------+----+
|           Car_Name|Fuel_Type|Year|
+-------------------+---------+----+
|                800|   Petrol|2003|
|                sx4|   Petrol|2003|
|            corolla|   Petrol|2004|
|Hero Super Splendor|   Petrol|2005|
|             innova|   Diesel|2005|
|             innova|   Petrol|2005|
|             innova|   Petrol|2005|
|            wagon r|   Petrol|2006|
|              camry|   Petrol|2006|
|   Bajaj Pulsar 150|   Petrol|2006|
+-------------------+---------+----+
only showing top 10 rows



In [70]:
# filtering with name using not-in
spark.sql("select Car_Name,Fuel_Type,Year from car_dataset order by Year desc").show(10)


+--------------------+---------+----+
|            Car_Name|Fuel_Type|Year|
+--------------------+---------+----+
|       vitara brezza|   Diesel|2018|
|   Bajaj Avenger 220|   Petrol|2017|
|       corolla altis|   Petrol|2017|
|  UM Renegade Mojave|   Petrol|2017|
|                ciaz|   Petrol|2017|
|           KTM RC200|   Petrol|2017|
|               ignis|   Petrol|2017|
|   Bajaj Dominar 400|   Petrol|2017|
|Royal Enfield Cla...|   Petrol|2017|
|            fortuner|   Diesel|2017|
+--------------------+---------+----+
only showing top 10 rows



#### use of count

In [81]:
# counting with condition
spark.sql("select count(*) as item_count from car_dataset where Year==2015 ").show()


+----------+
|item_count|
+----------+
|        61|
+----------+

